# Математические модели и формулы

<font color='red'>TODO:</font> Заменить скобки на нормальные

# Обозначения

* $G = (V,E) $ — граф
* $A \in \mathbb{R}^{N\times N}$ — матрица смежности 
* $F \in \mathbb{R}^{N\times K}$ — матрица принадлежности к сообществам
* $K$ — количество сообществ
* $C$ — множество сообществ


## BigCLAM

![title](imgs/BigCLAM_model.png)

### Предположения

* Каждая вершина $v$ относится к сообществу $c$ с некоторой силой $F_{vc} >0$. 
* Вероятность появления ребра $(u,v)$, при условии, что вершины $u,v$ находятся в одном сообществе $c$ есть 

$$P((u,v) | c)=1 - \exp(-F_{uc} F_{vc}).$$

* Каждое сообщество $c$ генерирует ребра независимо друг от друга, а значит, что вероятность появления ребра 

$$P(u,v)=1 - \exp(-\sum_{c\in C} F_{uc} F_{vc}) = 1 - \exp( - F_{u} F_{v}^T),$$

$$F = \{F_u\} = \{F_{uc}\} \in \mathbb{R}^{N \times K}. $$


### Вероятностная интерпретация

* Каждые две вершины $u, v$ взаимодействуют с некоторой силой $X_{uv}$. Чем больше сила, тем больше вероятность $p(u,v)$ появления ребра.
* Сила взаимодействия вершин $u, v$ определяется сообществами. Каждое сообщество $c$, в которое входит одновременно 2 вершины дает свой аддитивный вклад в силу их взаимодействия $X_{uv}^{(c)}$.
* **Предполагаем**, что $X_{uv}^{(c)} \sim \mathrm{Pois}(F_{uc} \cdot F_{vc})$, где $F_{vc}>0$ сила взаимодействия вершины $v$ и сообщества $c$. Значит, что 

$$X_{uv} \sim \mathrm{Pois}(\sum_{c} F_{uc} \cdot F_{vc}) = \mathrm{Pois}(F_{u} \cdot F_{v}^T).$$

* **Предполагаем**, что ребро появляется, если $X_{uv} > 0$. Т.е.

$$p(u,v) = \mathbb{P}(X_{uv} > 0) = 1 - \exp( - F_{u} F_{v}^T).$$

### $\varepsilon$-сообщество
Предполагаем, что все вершины относятся к большому $\varepsilon$-сообществу с малой силой ($\approx 10^{-6}$) (т.к. иначе, вершины, не входящие в одно сообщество не могут быть соединены ребром). 

### Модель

Используется метод максимизации правдоподобия.

$$l(F) = log(\mathbb{P}(A|F))$$

$$l(F) = \sum_{(u,v)\in E} \log(1 - \exp( - F_{u} F_{v}^T)) - \sum_{(u,v) \notin E} F_u F_v^T$$

<font color='red'>Вопрос:</font> не портят ли наши регуляризации выпуклости?

### Схема оптимизации

Такая задача является частным случаем NMF (Non-negative matrix factorization):
Ищем такую низкоранговую матрицу $F\in \mathbb{R}^{N \times K}$, что она наилучшим образом приближает матрицу $A$ в смысле правдоподобия (на самом деле $l_2$-норма плохо подходит для восстановления бинарных матриц): 

$$ \hat{F} = \arg \min_{F \ge 0} D(A, f(FF^T)),$$

$$\text{гдe}\quad D = -l(F), \quad f(x) = 1-\exp(-x).$$



Для оптимизации используется блочный координатный спуск с методом проекции градиента на каждом шаге.
Фиксируем $F_v$, оптимизируем по $F_u$, $u \ne v$. Задача становится выпуклой.

$$\forall u: \quad \arg\max_{F_u \ge 0} l(F_u), $$

$$l(F_u) = \sum_{v \in \mathcal{N}(u)} \log(1-\exp(-F_u F_v^T)) - \sum_{v \notin \mathcal{N}(u)} F_u F_v^T $$

$\mathcal{N}(u)$ — соседи вершины u.

$$\nabla l(F_u) = \sum_{v \in \mathcal{N}(u)} F_u \dfrac{\exp(-F_u F_v^T)}{1-exp(-F_u F_v^T)} - \sum_{v \notin \mathcal{N}(u)} F_v^T. $$

Основная сложность формулы (линейная по размеру графа) во втором слагаемом. Заметим, что 

$$\sum_{v \notin \mathcal{N}(u)} F_v^T = \sum_v{F_v} - F_u - \sum_{v\in \mathcal{N}(u)} F_v.$$ 

Получаем сложность одной итерации $O(\mathcal{N}(u))$.

Для подбора градиентного шага используем backtracking line search.

### Восстановление структуры сообществ

Для того, чтобы восстановить исходную структуру сообществ $C$ сравним значение матрицы $F$ с порогом $\delta$. Обозначим за $\varepsilon$ вероятность появления ребра в графе (если бы все ребра появлялись равномерно): $\varepsilon = \dfrac{2|V|}{|E|\cdot (|E|-1)}$. Возьмем $\delta$ так, чтобы две вершины принадлежали одному сообществу, если модельная вероятность появления ребра между ними выше чем $\varepsilon$:

$$\varepsilon \le 1-\exp(-\delta^2)$$

$$\delta = \sqrt{-\log(1-\varepsilon)} $$

### Инициализация

<font color='red'>TODO:</font> Заполнить

## WeiCLAM

Самое простое изменение BigCLAM для обработки взвешенных ребер:

$$l(F) = \sum_{(u,v)\in E} \log(1 - \exp\left( - \dfrac{F_{u} F_{v}^T}{w_{uv}}\right) - \sum_{(u,v) \notin E} \dfrac{F_{u} F_{v}^T}{w_{uv}}.$$

Тем самым, мы получаем, что чем больше вес $w_{uv}$, тем больше должно быть значение сил $F_u$ и $F_v$, которые его объясняют.

Т.е. Предположение, что вероятность появления ребра $(u,v)$, при условии, что вершины $u,v$ находятся в одном сообществе $c$ есть 

$$P((u,v) | c)=1 - \exp\left( -F_{uc} F_{vc} \right).$$

Заменяется на предположение, что вероятность появления ребра $(u,v)$ **с весом $w_{uv}$**, при условии, что вершины $u,v$ находятся в одном сообществе $c$ есть 

$$P((u,v) | c, w_{uv})=1 - \exp\left(-\dfrac{F_{uc} F_{vc}}{w_{uv}}\right).$$

<font color='red'>TODO:</font> Дописать.

Красивая вероятностная интерпретация: $$X_{uv}^{(c)} \sim \mathrm{Pois}\left(\dfrac{F_{uc} \cdot F_{vc}}{w_{uv}}\right)$$

Тесты подтверждают работоспособность модели.

## GammaModel

Совершенно другая вероятностная модель к описанию наблюдаемых графов.

### Предпосылки

тут еще есть про "правильный" аналог непрерывного распределения Пуассона: http://ac.inf.elte.hu/Vol_039_2013/137_39.pdf
надо попробовать потренировать ее тоже.

Если мы посмотрим на первоначальную модель, то увидим, что в ней есть скрытые переменные $X_{uv}$, которые распределены по Пуассону. Данное распределение является дискретным, а веса на ребрах -- непрерывные. 

Самое главное свойство, которые использовались в выводе — мультипликативность. Так что в качестве непрерывного аналога распределения Пуассона можно взять Гамма распределение, сумма которых (с одинаковым коэффициентом масштаба) не выводит из класса.

### Предположения

* Вероятность появления ребра с весом $w_{uv}$, при условии, что вершины принадлежат сообществу $c$

$$p\left(w_{uv} | c\right) \sim \mathrm{\Gamma}\left(k=F_u F_v^T, \theta=1\right).$$

* Каждое сообщество $c$ генерирует ребра независимо друг от друга, а значит, что вес ребра

$$w_{uv} = \sum_{c} w_{uv}^c \sim \mathrm{\Gamma}\left(\sum_c F_{uc} F_{vc} + 1, 1\right) = \mathrm{\Gamma}\left(F_u F_v^T + 1, 1\right).$$

Берем $+ 1$ для того, чтобы не сталкиваться с распределениями с бесконечной плотностью в нуле. В вероятностном смысле это безусловная (от сообществ) вероятность появления ребра в графе.


Для простоты везде далее будем опускать параметр $\theta$.

Обозначим $K_{uv} = F_u F_v^T + 1.$

### Модель

$$l\left(F\right) = log\left(\mathbb{P}\left(A|F\right)\right)$$

$$ l\left(F\right) = \sum_{w_uv} \log p(w_{uv}) $$

$$ l\left(F\right)= \sum_{w_{uv}}\left[-\log\mathrm{\Gamma}\left(K_{uv}\right) - K_{uv}\log\theta + (K_{uv} - 1)\cdot\log w_{uv} - \dfrac{w_{uv}}{\theta}\right] $$

$$ l\left(F\right)= \sum_{w_{uv}}\left[-\log\mathrm{\Gamma}\left(K_{uv}\right) + (K_{uv} - 1)\cdot\log w_{uv} - w_{uv}\right] $$

$$ l\left(F\right)= \sum_{w_{uv}}\left[-\log\mathrm{\Gamma}\left( F_u F_v^T + 1 \right) + F_u F_v^T \cdot\log w_{uv} - w_{uv}\right] \rightarrow \max_{F\ge 1}.$$

### Схема оптимизации

Для того, чтобы посчитать градиент, нам понадобится дигамма функция:

$$\Psi(x) = \dfrac{\mathrm{d}}{\mathrm{d}x} \log\left(\mathrm\Gamma(x)\right)$$

$$\dfrac{\mathrm{d}l(F)}{\mathrm{d}F_u} = - \sum_v F_v \Psi\left(F_u F_v^T + 1\right) + F_v \cdot  \left( \log\theta - \log w_{uv}\right)$$

$$\dfrac{\mathrm{d}l(F)}{\mathrm{d}F_u} = - \sum_v F_v \Psi\left(F_u F_v^T + 1\right) - F_v \log w_{uv}.$$

Схема оптимизации используется та же самая, что и в BigCLAM.

Ко всем весам прибавляется небольшое $\varepsilon$, чтобы избежать нулей под логарифмом.

Для ускорения вычисления можно отдельно хранить $\log w_{uv}$ и $\Psi\left(F_u F_v^T + 1\right)$, каждый раз обновляя 1 строчку и 1 столбец. Тем не менее, т.к. сумма взвешенная, провести такой трюк как в BigCLAM не получится. Для каждого шага, для каждого $F_u$ придется пересчитывать сумму целиком. Получается линейная сложность.

<font color='red'>TODO:</font> Оптимизировать схему оптимизации в коде.

### Инициализация

<font color='red'>TODO:</font> Понять, что подход из BigClam обобщается и на этот случай, записать как, а еще можно и теоремы передоказать (например, конкретно для этой модели).